I loved simple bank. It was the easies banking interface ever. And it geared to help me save money not spend it, lol.

Sadly, they're gone. So, here's my initial crack at making one.

In [1]:
import pandas as pd
import numpy as np

In [39]:
#import datetime as dt     
today = pd.to_datetime('today').normalize()

In [40]:
print(today)

2021-08-09 00:00:00


In [2]:
# first i'll need a calendar.

def create_date_table(start='2000-01-01', end='2050-12-31'):
    df = pd.DataFrame({"Date": pd.date_range(start, end)})
    df["Day"] = df.Date.dt.day_name()
    df["Week"] = df.Date.dt.weekofyear
    df["Quarter"] = df.Date.dt.quarter
    df["Year"] = df.Date.dt.year
    df["Year_half"] = (df.Quarter + 1) // 2
    return df


In [3]:
df_cal = create_date_table('2021-08-01', '2021-12-31')

<ipython-input-2-8be5acf6b006>:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df["Week"] = df.Date.dt.weekofyear


In [4]:
df_cal.head()

,Date,Day,Week,Quarter,Year,Year_half
0,2021-08-01,Sunday,30,3,2021,2
1,2021-08-02,Monday,31,3,2021,2
2,2021-08-03,Tuesday,31,3,2021,2
3,2021-08-04,Wednesday,31,3,2021,2
4,2021-08-05,Thursday,31,3,2021,2


In [158]:
def calculate_paydays(df_cal, payday = '2021-08-11', frequency = 2, income = 3000):
    #payday = input("When's your next payday? ")
    #frequency = input("How often do you get paid? (enter 1 for weekly, 2 for bi-weekly, 3 for monthy, 4 for twice monthly) ")
    #df_cal['payday'] = np.where(df_cal['Date'] == payday, 'payday', '')
    #income = input("How much money do you get each payday? ")
    start = df_cal.index[df_cal['Date'] == payday].tolist()
    paydays = df_cal[df_cal.index % 14 == start[0]].index
    df_cal['payday'] = np.where(df_cal.index.isin(paydays), 'payday', np.NaN)
    df_cal['income'] = np.where(df_cal.index.isin(paydays), income, np.NaN)
    df_cal['set_aside'] = np.NaN

In [159]:
calculate_paydays(df_cal)

In [161]:
df_cal.head()

,Date,Day,Week,Quarter,Year,Year_half,payday,income,set_aside
0,2021-08-01,Sunday,30,3,2021,2,nan,NaN,NaN
1,2021-08-02,Monday,31,3,2021,2,nan,NaN,NaN
2,2021-08-03,Tuesday,31,3,2021,2,nan,NaN,NaN
3,2021-08-04,Wednesday,31,3,2021,2,nan,NaN,NaN
4,2021-08-05,Thursday,31,3,2021,2,nan,NaN,NaN


In [165]:
bills = pd.read_excel('input_list.xlsx', sheet_name = 'bills')

In [167]:
bills['save_per_day'] = np.NaN

In [168]:
def set_asides(bills, df_cal):
    for index, row in bills.iterrows():
        due_date = row['due_date']
        paydays_between = df_cal[(df_cal.payday == 'payday')
                                 & (df_cal.Date > today)
                                 & (df_cal.Date < due_date)
                                ].payday.count()
        savings_per_payday = row['amount'] / paydays_between
        #print(savings_per_payday)
        #print(index)
        bills.loc[index,'save_per_day'] = savings_per_payday

In [169]:
set_asides(bills, df_cal)

In [170]:
bills.head()

,category,name,due_date,end_date,amount,frequency,save_per_day
0,bill,verizon,2021-09-13,NaN,130,monthly,43.333333
1,bill,electric,2021-09-27,NaN,100,monthly,25.000000
2,bill,crossfit,2021-09-15,NaN,150,monthly,50.000000


In [173]:
# now take the bills data-frame complete with 
for index, row in df_cal.iterrows():
    if row['payday'] == 'payday':
        date = row['Date']
        set_aside = bills[['due_date', 'save_per_day']].sum(axis=1).where(bills['due_date'] > date, 0)
        df_cal.loc[index, 'set_aside'] = set_aside[0]
    else: pass
    

In [176]:
#df[['A','B']].sum(axis=1).where(df['A'] > 0, 0)
bills[['due_date', 'save_per_day']].sum(axis=1).where(bills['due_date'] > '2021-09-23', 0)

0     0.0
1    25.0
2     0.0
dtype: float64

In [177]:
df_cal.head(20)

,Date,Day,Week,Quarter,Year,Year_half,payday,income,set_aside
0,2021-08-01,Sunday,30,3,2021,2,nan,NaN,NaN
1,2021-08-02,Monday,31,3,2021,2,nan,NaN,NaN
2,2021-08-03,Tuesday,31,3,2021,2,nan,NaN,NaN
3,2021-08-04,Wednesday,31,3,2021,2,nan,NaN,NaN
4,2021-08-05,Thursday,31,3,2021,2,nan,NaN,NaN
5,2021-08-06,Friday,31,3,2021,2,nan,NaN,NaN
6,2021-08-07,Saturday,31,3,2021,2,nan,NaN,NaN
7,2021-08-08,Sunday,31,3,2021,2,nan,NaN,NaN
8,2021-08-09,Monday,32,3,2021,2,nan,NaN,NaN
9,2021-08-10,Tuesday,32,3,2021,2,nan,NaN,NaN
